In [17]:
import requests
import os
from dotenv import load_dotenv, find_dotenv
from goodreads import client
import goodreads
import codecs
from kitchen.text.converters import to_unicode
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

# TODO, fix that stupid unicode/utf-8 issue
# TODO, if author check fails, do a different search

GOODREADS_KEY    = os.environ.get('GOODREADS_KEY')
GOODREADS_SECRET = os.environ.get('GOODREADS_SECRET') 

filename = '../data/locus_2018.csv'
FAILED_PATH = '../results/failed.tmp.txt'
SCORED_PATH = '../results/scored.tmp.txt'

gc = client.GoodreadsClient(GOODREADS_KEY, GOODREADS_SECRET)

def rank_lookup(book):
    '''search_books(self, q, page=1, search_field='all')
     |      Get the most popular books for the given query. This will search all
     |      books in the title/author/ISBN fields and show matches, sorted by
     |      popularity on Goodreads.
     |      :param q: query text
     |      :param page: which page to return (default 1)
     |      :param search_fields: field to search, one of 'title', 'author' or
     |      'genre' (default is 'all')
    '''
    searchterm = book['title'].lstrip()
    returnvalue = gc.search_books(searchterm,page=1,search_field='title')
    return returnvalue[0].average_rating,returnvalue[0].authors,returnvalue[0].publication_date

def read_input(filename):
    with open(filename,"r") as fh:
        data = fh.readlines()
    return data

def recode(string):
    description = to_unicode(string, 'utf-8')
    return description

def parse_file(data):
    booklist = []
    for line in data:
        print (line)
        if "(" in line:
            title = recode(line.split(",")[0])
            author = recode(line.split(",")[1].split("(")[0])
            booklist.append({'title':title,'author':author})
    return booklist

def save_list(booklist,PATH):
    with open(PATH,'w') as outfile:
        for book in booklist:
            outfile.write(book + '\n')

def lookup_entry(booklist):
    scored_books = []; failed_books = []
    for i,book in enumerate(booklist):
        rank,checkauthor,pubdate = rank_lookup(book)
        try:
            output_string =  ','.join([str(i), rank, book['title'].lstrip(), book['author'],\
                                       str(checkauthor[0]), pubdate[2] ])
            print (output_string)
            scored_books.append(output_string)
        except:
            output_string =  ','.join([str(i), 'failed-rank', book['title'].lstrip(), book['author'],\
                                       'failed-author', 'failed-date'])
            print (output_string)
            scored_books.append(output_string)
            failed_books.append(output_string)
            
    save_list(failed_books,FAILED_PATH)
    save_list(scored_books,SCORED_PATH)
    
    return scored_books,failed_books
        
data = read_input(filename)
booklist = parse_file(data)
scored_books, failed_books = lookup_entry(booklist)

Record of a Spaceborn Few, Becky Chambers (Harper Voyager US; Hodder & Stoughton)

Emergence, C.J. Cherryh (DAW)

Chercher La Femme, L. Timmel Duchamp (Aqueduct)

Europe at Dawn, Dave Hutchinson (Solaris UK)

Shelter, Dave Hutchinson (Solaris US; Solaris UK)

Pride and Prometheus, John Kessel (Saga)

The Calculating Stars, Mary Robinette Kowal (Tor)

If Tomorrow Comes, Nancy Kress (Tor)

Revenant Gun, Yoon Ha Lee (Solaris US; Solaris UK)

Blackfish City, Sam J. Miller (Ecco; Orbit UK)

Embers of War, Gareth L. Powell (Titan US; Titan UK)

The Overstory, Richard Powers (Norton; Heinemann)

The Dreaming Stars, Tim Pratt (Angry Robot US; Angry Robot UK)

An American Story, Christopher Priest (Gollancz)

Elysium Fire, Alastair Reynolds (Gollancz; Orbit US)

By the Pricking of Her Thumb, Adam Roberts (Gollancz)

Red Moon, Kim Stanley Robinson (Orbit US; Orbit UK)

Theory of Bastards, Audrey Schulman (Europa)

The Sky Is Yours, Chandler Klang Smith (Hogarth US)

Unholy Land, Lavie Tidhar (Ta

53,4.33,The Hunger, Alma Katsu ,Suzanne Collins,2008
54,3.99,Providence, Caroline Kepnes ,Jamie McGuire,2010
55,4.08,The Outsider, Stephen King ,S.E. Hinton,1997
56,4.11,The Listener, Robert McCammon ,Robert R. McCammon,2018
57,3.70,Cross Her Heart, Sarah Pinborough ,Sarah Pinborough,2018
58,3.35,The Cabin at the End of the World, Paul Tremblay ,Paul Tremblay,2018
59,failed-rank,Tide of Stone, Kaaron Warren ,failed-author,failed-date
60,3.50,Damsel, Elana K. Arnold ,Elana K. Arnold,2018
61,3.71,The Gone Away Place, Christopher Barzak ,Christopher Barzak,2018
62,4.18,The Cruel Prince, Holly Black ,Holly Black,2018
63,3.70,The Brilliant Death, Amy Rose Capetta ,Amy Rose Capetta,2018
64,3.88,The Belles, Dhonielle Clayton ,Dhonielle Clayton,2018
65,3.94,Tess of the Road, Rachel Hartman ,Rachel Hartman,2018
66,3.79,For a Muse of Fire, Heidi Heilig ,Heidi Heilig,2018
67,4.17,Dread Nation, Justina Ireland ,Justina Ireland,2018
68,4.06,Cross Fire, Fonda Lee ,James Patterson,2010
69,3.74,Black 

FileNotFoundError: [Errno 2] No such file or directory: '../results/failed.tmp.txt'